### This workflow comes from the 'Moving Pictures' tutorial.

#### Prepare

In [ ]:
# Create qiime2 virtual env
wget https://data.qiime2.org/distro/core/qiime2-2020.2-py36-linux-conda.yml
conda env create -n qiime2-2020.2 --file qiime2-2020.2-py36-linux-conda.yml
rm qiime2-2020.2-py36-linux-conda.yml

In [ ]:
# Activate qiime2 virtual env
source activate qiime2-2020.2

In [ ]:
# Create working directory
mkdir qiime2-moving-pictures-tutorial
cd qiime2-moving-pictures-tutorial

In [ ]:
# Download metadata 
wget -O "sample-metadata.tsv" "https://data.qiime2.org/2019.7/tutorials/moving-pictures/sample_metadata.tsv"

In [ ]:
# Download seq data
mkdir -p emp-single-end-sequences
wget -O "emp-single-end-sequences/barcodes.fastq.gz" "https://data.qiime2.org/2019.7/tutorials/moving-pictures/emp-single-end-sequences/barcodes.fastq.gz"
wget -O "emp-single-end-sequences/sequences.fastq.gz" "https://data.qiime2.org/2019.7/tutorials/moving-pictures/emp-single-end-sequences/sequences.fastq.gz"

In [ ]:
# Generate qiime2 required artifact
# The semantic type of this QIIME 2 artifact is EMPSingleEndSequences.
# EMPSingleEndSequences QIIME 2 artifacts contain sequences that are multiplexed, meaning that the sequences have not yet been assigned to samples (hence the inclusion of both sequences.fastq.gz and barcodes.fastq.gz files, where the barcodes.fastq.gz contains the barcode read associated with each sequence in sequences.fastq.gz.) 
qiime tools import \
  --type EMPSingleEndSequences \
  --input-path emp-single-end-sequences \
  --output-path emp-single-end-sequences.qza

#### Demultiplexing sequences

#### Essentially, this step is 'QC'.

In [ ]:
# To know which barcode sequence is associated with each sample
# This information is contained in the sample metadata file.
qiime demux emp-single \
  --i-seqs emp-single-end-sequences.qza \
  --m-barcodes-file sample-metadata.tsv \
  --m-barcodes-column barcode-sequence \
  --o-per-sample-sequences demux.qza \
  --o-error-correction-details demux-details.qza
# The demux.qza QIIME 2 artifact will contain the demultiplexed sequences. 
# The second output (demux-details.qza) presents Golay error correction details.

In [ ]:
# To generate a summary of the demultiplexing results.
# To determine how many sequences were obtained per sample.
# To get a summary of the distribution of sequence qualities at each position in your sequence data.
qiime demux summarize \
  --i-data demux.qza \
  --o-visualization demux.qzv

#### Sequence quality control and feature table construction

##### QIIME 2 plugins are available for several quality control methods, including DADA2 and Deblur.

##### The result of both of these methods will be a FeatureTable[Frequency] QIIME 2 artifact, which contains counts (frequencies) of each unique sequence in each sample in the dataset, and a FeatureData[Sequence] QIIME 2 artifact, which maps feature identifiers in the FeatureTable to the sequences they represent..

##### First, we try DADA2.
##### DADA2 is a pipeline for detecting and correcting (where possible) Illumina amplicon sequence data. 

In [ ]:
qiime dada2 denoise-single \
  --i-demultiplexed-seqs demux.qza \
  --p-trim-left 0 \
  --p-trunc-len 120 \
  --o-representative-sequences rep-seqs-dada2.qza \
  --o-table table-dada2.qza \
  --o-denoising-stats stats-dada2.qza

# For qiime2-view
qiime metadata tabulate \
  --m-input-file stats-dada2.qza \
  --o-visualization stats-dada2.qzv

# If you’d like to continue the tutorial using this FeatureTable (opposed to the Deblur feature table), run the following commands.
mv rep-seqs-dada2.qza rep-seqs.qza
mv table-dada2.qza table.qza

##### Then, we try Deblur.
##### Deblur uses sequence error profiles to associate erroneous sequence reads with the true biological sequence from which they are derived, resulting in high quality sequence variant data.

In [ ]:
qiime quality-filter q-score \
 --i-demux demux.qza \
 --o-filtered-sequences demux-filtered.qza \
 --o-filter-stats demux-filter-stats.qza

qiime deblur denoise-16S \
  --i-demultiplexed-seqs demux-filtered.qza \
  --p-trim-length 120 \
  --o-representative-sequences rep-seqs-deblur.qza \
  --o-table table-deblur.qza \
  --p-sample-stats \
  --o-stats deblur-stats.qza

# For qiime2-view
qiime metadata tabulate \
  --m-input-file demux-filter-stats.qza \
  --o-visualization demux-filter-stats.qzv

qiime deblur visualize-stats \
  --i-deblur-stats deblur-stats.qza \
  --o-visualization deblur-stats.qzv

# If you’d like to continue the tutorial using this FeatureTable (opposed to the DADA2 feature table), run the following commands.
mv rep-seqs-deblur.qza rep-seqs.qza
mv table-deblur.qza table.qza

#### FeatureTable and FeatureData summaries

#### Essentitally, this step is 'summary'.

In [ ]:
# To give you information on how many sequences are associated with each sample and with each feature, histograms of those distributions, and some related summary statistics.
qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file sample-metadata.tsv

# To give you information on how many sequences are associated with each sample and with each feature, histograms of those distributions, and some related summary statistics.
qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv

#### Generate a tree for phylogenetic diversity analyses

In [ ]:
qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

#### Alpha and beta diversity analysis

In [ ]:
# To rarefy a FeatureTable[Frequency] to a user-specified depth, computes several alpha and beta diversity metrics, and generates principle coordinates analysis (PCoA) plots using Emperor for each of the beta diversity metrics.
# The metrics computed by default are:
# Alpha diversity:
# Shannon’s diversity index (a quantitative measure of community richness)
# Observed OTUs (a qualitative measure of community richness)
# Faith’s Phylogenetic Diversity (a qualitiative measure of community richness that  incorporates phylogenetic relationships between the features)
# Evenness (or Pielou’s Evenness; a measure of community evenness)
# Beta diversity
# Jaccard distance (a qualitative measure of community dissimilarity)
# Bray-Curtis distance (a quantitative measure of community dissimilarity)
# unweighted UniFrac distance (a qualitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)
# weighted UniFrac distance (a quantitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)

qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table table.qza \
  --p-sampling-depth 1103 \
  --m-metadata-file sample-metadata.tsv \
  --output-dir core-metrics-results

# Output artifacts:
# core-metrics-results/faith_pd_vector.qza
# core-metrics-results/unweighted_unifrac_distance_matrix.qza
# core-metrics-results/bray_curtis_pcoa_results.qza
# core-metrics-results/shannon_vector.qza
# core-metrics-results/rarefied_table.qza
# core-metrics-results/weighted_unifrac_distance_matrix.qza
# core-metrics-results/jaccard_pcoa_results.qza
# core-metrics-results/observed_otus_vector.qza
# core-metrics-results/weighted_unifrac_pcoa_results.qza
# core-metrics-results/jaccard_distance_matrix.qza
# core-metrics-results/evenness_vector.qza
# core-metrics-results/bray_curtis_distance_matrix.qza
# core-metrics-results/unweighted_unifrac_pcoa_results.qza
# Output visualizations:
# core-metrics-results/unweighted_unifrac_emperor.qzv
# core-metrics-results/jaccard_emperor.qzv
# core-metrics-results/bray_curtis_emperor.qzv
# core-metrics-results/weighted_unifrac_emperor.qzv

In [ ]:
# To perform perform pair-wise comparison for alpha diversity
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization core-metrics-results/faith-pd-group-significance.qzv

qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/evenness_vector.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization core-metrics-results/evenness-group-significance.qzv

In [ ]:
# To perform pair-wise comparison for beta diversity
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column body-site \
  --o-visualization core-metrics-results/unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column subject \
  --o-visualization core-metrics-results/unweighted-unifrac-subject-group-significance.qzv \
  --p-pairwise

In [ ]:
# To explore how these samples changed over time.
qiime emperor plot \
  --i-pcoa core-metrics-results/unweighted_unifrac_pcoa_results.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-custom-axes days-since-experiment-start \
  --o-visualization core-metrics-results/unweighted-unifrac-emperor-days-since-experiment-start.qzv

qiime emperor plot \
  --i-pcoa core-metrics-results/bray_curtis_pcoa_results.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-custom-axes days-since-experiment-start \
  --o-visualization core-metrics-results/bray-curtis-emperor-days-since-experiment-start.qzv

#### Alpha rarefaction plotting

In [ ]:
qiime diversity alpha-rarefaction \
  --i-table table.qza \
  --i-phylogeny rooted-tree.qza \
  --p-max-depth 4000 \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization alpha-rarefaction.qzv

#### Taxonomic analysis

In [ ]:
# To explore the taxonomic composition of the samples, and again relate that to sample metadata. 
# The first step in this process is to assign taxonomy to the sequences in our FeatureData[Sequence] QIIME 2 artifact by using a pre-trained Naive Bayes classifier and the q2-feature-classifier plugin
# This classifier was trained on the Greengenes 13_8 99% OTUs, where the sequences have been trimmed to only include 250 bases from the region of the 16S that was sequenced in this analysis (the V4 region, bound by the 515F/806R primer pair)..

# Download classifier
wget \
  -O "gg-13-8-99-515-806-nb-classifier.qza" \
  "https://data.qiime2.org/2020.2/common/gg-13-8-99-515-806-nb-classifier.qza"

qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy.qza

qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

# For qiime2-view
qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization taxa-bar-plots.qzv

#### Differential abundance testing with ANCOM

##### ANCOM can be applied to identify features that are differentially abundant (i.e. present in different abundances) across sample groups.

In [ ]:
# To create a feature table that contains only the gut samples.
qiime feature-table filter-samples \
  --i-table table.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-where "[body-site]='gut'" \
  --o-filtered-table gut-table.qza

# To aviod frequencies of zero in FeatureTable[Composition].
qiime composition add-pseudocount \
  --i-table gut-table.qza \
  --o-composition-table comp-gut-table.qza

# Run ANCOM on the subject column to determine what features differ in abundance across the gut samples of the two subjects.
qiime composition ancom \
  --i-table comp-gut-table.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column subject \
  --o-visualization ancom-subject.qzv

In [ ]:
# To performe a differential abundance test at a specific taxonomic level. 
# To do this, we can collapse the features in our FeatureTable[Frequency] at the taxonomic level of interest, and then re-run the above steps.
qiime taxa collapse \
  --i-table gut-table.qza \
  --i-taxonomy taxonomy.qza \
  --p-level 6 \
  --o-collapsed-table gut-table-l6.qza

qiime composition add-pseudocount \
  --i-table gut-table-l6.qza \
  --o-composition-table comp-gut-table-l6.qza

qiime composition ancom \
  --i-table comp-gut-table-l6.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column subject \
  --o-visualization l6-ancom-subject.qzv